In [ ]:
import os
import torch
import librosa
from itertools import product  # Pour générer toutes les paires possibles
import torchaudio
import torchaudio.transforms as T

cocola_path = "/home/a23marmo/Desktop/Projects/Project Releases/AutoMashup/cocola" # To change with you path to the cocola model
import sys
sys.path.append(cocola_path)
from contrastive_model import constants
from contrastive_model.contrastive_model import CoCola
from feature_extraction.feature_extraction import CoColaFeatureExtractor

# Dossier contenant les fichiers séparés (vocals, bass, drums, other)
htdemucs_folder = "/home/a23marmo/Desktop/Projects/Project Releases/AutoMashup/git_axle/automashup/data/separated/htdemucs" # To change with your path to the data folder of automashup

# Charger le modèle pré-entraîné Cocola
# Charger le modèle pré-entraîné Cocola avec des entrées à 2 canaux
model = CoCola.load_from_checkpoint(
    f"{cocola_path}/checkpoint.ckpt", 
    input_type=constants.ModelInputType.DOUBLE_CHANNEL_HARMONIC_PERCUSSIVE
)

feature_extractor = CoColaFeatureExtractor()
model.eval()

# Définir le mode d'embedding
model.set_embedding_mode(constants.EmbeddingMode.BOTH)  # Ou HARMONIC/PERCUSSIVE selon besoin

# Stockage des features extraits pour éviter de les recalculer
vocals_features = {}
instrumentals_features = {}

# Fonction pour charger et prétraiter un fichier audio
def load_and_preprocess(file_path, target_sr=44100):
    print(f"📥 Chargement de {file_path}")  # Debug
    waveform, base_sr = torchaudio.load(file_path)

    # Resample à la fréquence cible si nécessaire
    if base_sr != target_sr:
        resampler = T.Resample(base_sr, target_sr)
        waveform = resampler(waveform)

    # Convertir en mono si nécessaire
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=False)
    else:
        waveform = waveform.squeeze()

    # Tronquer à 10 secondes maximum (si nécessaire)
    #max_length = target_sr * 10  # 10 secondes
    #waveform = waveform[:max_length]

    # Ajouter une dimension de batch et une dimension de canal
    waveform = waveform.unsqueeze(0)  # (B, C, N)
    return waveform
# **Parcourir tous les dossiers des musiques pour extraire les embeddings**
for music_folder in os.listdir(htdemucs_folder):
    music_path = os.path.join(htdemucs_folder, music_folder)
    if not os.path.isdir(music_path):
        continue  # Ignorer les fichiers non pertinents

    print(f"\n🔹 Extraction des features pour : {music_folder}")

    vocals_path = os.path.join(music_path, "vocals.wav")
    bass_path = os.path.join(music_path, "bass.wav")
    drums_path = os.path.join(music_path, "drums.wav")
    other_path = os.path.join(music_path, "other.wav")

    # Charger et traiter les vocaux
    if os.path.exists(vocals_path):
        vocals_waveform = load_and_preprocess(vocals_path)
        vocals_features[music_folder] = feature_extractor(vocals_waveform)

    # Additionner les fichiers instrumentaux
    instrumental_waveform = None
    for inst_path in [bass_path, drums_path, other_path]:
        if os.path.exists(inst_path):
            waveform = load_and_preprocess(inst_path)
            instrumental_waveform = waveform if instrumental_waveform is None else instrumental_waveform[:waveform.shape[0]] + waveform[:instrumental_waveform.shape[0]]

    # Stocker les features des instrumentaux
    if instrumental_waveform is not None:
        instrumentals_features[music_folder] = feature_extractor(instrumental_waveform)





In [ ]:
save_path = f"{cocola_path}/features_cocola"
os.makedirs(save_path, exist_ok=True)
# Sauvegarder les features extraites
torch.save(vocals_features, os.path.join(save_path, "vocals_features.pt"))
torch.save(instrumentals_features, os.path.join(save_path, "instrumentals_features.pt"))
print("\n✅ Extraction des features terminée !")